## Capstone Project The Battle of the Neighborhoods


## 1. Introduction: Business Problem 

This project is aimed at finding the optimal location to set a **Italian Reastaurant** in London,UK.

There are already many restaurants in London,we need to detect the **locations that did not already have restaurants**. Of course, not having too many **similar Italian restaurant is also important**.

Third, **the location should be as close to city center** as possible

## 2. Data

As what we intrduced before, there are four main factors to determine the location of the restaurant

1. number of restaurants existing in the neighborhoods

2. number of similar Italian restaurants in the neighborhoods

3. distance of neighborhoods from city center 

## Here are the data sources that needed to acquire the required information

1. the coordinate of the city centre can be acquired by using **geopy library**

2. number of restaurants and their type and other type of buildings can be obtained by **Foursquare API**

3. The neiborhoods data can also be acquired by Foursquare API

#### Use geopy library to get the latitude and longitude values of Beijing.

In [175]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests # library to handle requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

In [176]:
address = 'London, UK'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 51.5073219, -0.1276474.


### Use Four Square to get the neighborhoods

In [177]:

VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: FCO1UB1PJBWY1Z1OX0EVQU0BU0EWOD4T5DQW55BWJLMLJ4QL
CLIENT_SECRET:KCWZFCZEG0NL1FPBT3WPW5ZCETAY0L03QIZYW21VQPRTSETV


In [178]:
radius=1000
LIMIT=100

In [179]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=FCO1UB1PJBWY1Z1OX0EVQU0BU0EWOD4T5DQW55BWJLMLJ4QL&client_secret=KCWZFCZEG0NL1FPBT3WPW5ZCETAY0L03QIZYW21VQPRTSETV&ll=51.5073219,-0.1276474&v=20180605&radius=1000&limit=100'

In [180]:
import requests

In [181]:
results = requests.get(url).json()
'There are {} around City Centre.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around City Centre.'

In [182]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4ac518cdf964a520e6a520e3',
  'name': 'National Gallery',
  'location': {'address': 'Trafalgar Sq',
   'lat': 51.50887601013219,
   'lng': -0.1284778118133545,
   'labeledLatLngs': [{'label': 'display',
     'lat': 51.50887601013219,
     'lng': -0.1284778118133545}],
   'distance': 182,
   'postalCode': 'WC2N 5DN',
   'cc': 'GB',
   'city': 'London',
   'state': 'Greater London',
   'country': 'United Kingdom',
   'formattedAddress': ['Trafalgar Sq',
    'London',
    'Greater London',
    'WC2N 5DN',
    'United Kingdom']},
  'categories': [{'id': '4bf58dd8d48988d18f941735',
    'name': 'Art Museum',
    'pluralName': 'Art Museums',
    'shortName': 'Art Museum',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_art_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 

In [183]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
dataframe = json_normalize(items) 
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
dataframe_filtered

C:\Users\13652\anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,National Gallery,Art Museum,Trafalgar Sq,51.508876,-0.128478,"[{'label': 'display', 'lat': 51.50887601013219...",182,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...",NaN,NaN,4ac518cdf964a520e6a520e3
1,Trafalgar Square,Plaza,Trafalgar Sq,51.507987,-0.128048,"[{'label': 'display', 'lat': 51.507987, 'lng':...",79,WC2N 5DX,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...",NaN,NaN,4ac518cef964a520f9a520e3
2,Corinthia Hotel,Hotel,Whitehall Pl,51.506607,-0.124460,"[{'label': 'display', 'lat': 51.50660672347560...",234,SW1A 2BD,GB,London,Greater London,United Kingdom,"[Whitehall Pl, London, Greater London, SW1A 2B...",NaN,NaN,4d274945d86aa09024d41dc0
3,East Trafalgar Square Fountain,Fountain,Trafalgar Sq,51.508088,-0.127700,"[{'label': 'display', 'lat': 51.50808805163309...",85,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...",NaN,NaN,50aa7c98e4b0d042f9f0bd59
4,Barrafina,Spanish Restaurant,10 Adelaide St,51.509427,-0.125894,"[{'label': 'display', 'lat': 51.50942652693695...",263,WC2N 4HZ,GB,London,Greater London,United Kingdom,"[10 Adelaide St (William IV St), London, Great...",William IV St,NaN,53bab96d498e7e355fb53d6c
5,Trafalgar Square Lions,Outdoor Sculpture,Trafalgar Sq,51.507641,-0.127888,"[{'label': 'display', 'lat': 51.50764064540292...",39,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...",NaN,NaN,50aa6e60e4b078585157bbb5
6,Tandoor Chop House,North Indian Restaurant,8 Adelaide Street,51.509192,-0.125638,"[{'label': 'display', 'lat': 51.50919205694273...",250,WC2N 4HZ,GB,Charing Cross,Greater London,United Kingdom,"[8 Adelaide Street, Charing Cross, Greater Lon...",NaN,NaN,583c344181907921f51d0740
7,Gordon's Wine Bar,Wine Bar,47 Villiers St,51.507911,-0.123293,"[{'label': 'display', 'lat': 51.5079110929777,...",308,WC2N 6NE,GB,London,Greater London,United Kingdom,"[47 Villiers St (Watergate Walk), London, Grea...",Watergate Walk,NaN,4acdb591f964a520c7cc20e3
8,Horse Guards Parade,Plaza,The Household Division,51.504847,-0.126590,"[{'label': 'display', 'lat': 51.50484733116023...",285,SW1A 2AX,GB,London,Greater London,United Kingdom,"[The Household Division (Horse Guards Rd), Lon...",Horse Guards Rd,NaN,4ac518cdf964a520f0a520e3
9,Whitehall Gardens,Garden,Horseguards Ave,51.506354,-0.122900,"[{'label': 'display', 'lat': 51.50635373505395...",346,NaN,GB,London,Greater London,United Kingdom,"[Horseguards Ave, London, Greater London, Unit...",NaN,Westminster,4c0ce9597189c928961ed6b6


In [184]:
map_London = folium.Map(location=[latitude, longitude], zoom_start=13)
folium.Marker(location=[latitude, longitude], popup='London').add_to(map_London)
for lat, lon in zip(latitudes, longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_London) 
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_London

In [185]:
df_locations=dataframe_filtered[['address','distance','lat','lng']]
df_locations.loc[:,'address']=df_locations.loc[:,'address']+',London'

C:\Users\13652\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [186]:
df_locations.to_pickle('./locations.pkl')    

In [187]:
latitudes=df_locations['lat'].values.tolist()

In [188]:
longitudes=df_locations['lng'].values.tolist()

Here we get the restaurant near each neighbors

In [189]:
food_category = '4d4b7105d754a06374d81259' 
italian_restaurant_categories = ['4bf58dd8d48988d110941735','55a5a1ebe4b013909087cbb6','55a5a1ebe4b013909087cb7c',
                                 '55a5a1ebe4b013909087cba7','55a5a1ebe4b013909087cba1','55a5a1ebe4b013909087cba4',
                                 '55a5a1ebe4b013909087cb95','55a5a1ebe4b013909087cb89','55a5a1ebe4b013909087cb9b',
                                 '55a5a1ebe4b013909087cb98','55a5a1ebe4b013909087cbbf','55a5a1ebe4b013909087cb79',
                                 '55a5a1ebe4b013909087cbb0','55a5a1ebe4b013909087cbb3','55a5a1ebe4b013909087cb74',
                                 '55a5a1ebe4b013909087cbaa','55a5a1ebe4b013909087cb83','55a5a1ebe4b013909087cb8c',
                                 '55a5a1ebe4b013909087cb92','55a5a1ebe4b013909087cb8f','55a5a1ebe4b013909087cb86',
                                 '55a5a1ebe4b013909087cbb9','55a5a1ebe4b013909087cb7f','55a5a1ebe4b013909087cbbc',
                                 '55a5a1ebe4b013909087cb9e','55a5a1ebe4b013909087cbc2','55a5a1ebe4b013909087cbad']

In [190]:
def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def get_venues_near_location(lat, lon, category,  CLIENT_ID, CLIENT_SECRET, radius=1000, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
       CLIENT_ID, CLIENT_SECRET, VERSION, lat, lon, category, radius, LIMIT)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [191]:
import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, CLIENT_ID, CLIENT_SECRET, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            if is_res:
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_italian:
                    italian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, italian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
italian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_350.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, italian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('italian_restaurants_350.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Restaurant data loaded.


In [192]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 363
Total number of Italian restaurants: 45
Percentage of Italian restaurants: 12.40%
Average number of restaurants in neighborhood: 26.39


In [193]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('53bab96d498e7e355fb53d6c', 'Barrafina', 51.509426526936956, -0.1258944349847769, '10 Adelaide St (William IV St), London, Greater London, WC2N 4HZ, United Kingdom', 348, False)
('583c344181907921f51d0740', 'Tandoor Chop House', 51.50919205694273, -0.12563829447742592, '8 Adelaide Street, Charing Cross, Greater London, WC2N 4HZ, United Kingdom', 238, False)
('59b92a852955132234397fd2', 'Japan Centre', 51.509760861810946, -0.13127297164342394, '35b Panton St., London, Greater London, SW1Y 4EA, United Kingdom', 339, False)
('5d3ec947c96faa00082677b1', 'Nutshell', 51.509632, -0.126605, "30 Saint Martin's Lane, London, Greater London, WC2N 4ER, United Kingdom", 316, False)
('4ac518d7f964a5205aa820e3', 'J Sheekey', 51.510899, -0.127797, "28-32 St Martin's Court, Charing Cross, Greater London, WC2N 4AL, United Kingdom", 334, False)
('5b420a45b1ec13002cb58037', 'Bancone', 51.50952932810332, -0.1264335499444991, '39 William IV St, Covent Garden,

In [194]:
print('List of Italian restaurants')
print('---------------------------')
for r in list(italian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(italian_restaurants))

List of Italian restaurants
---------------------------
('5b420a45b1ec13002cb58037', 'Bancone', 51.50952932810332, -0.1264335499444991, '39 William IV St, Covent Garden, Greater London, WC2N 4DD, United Kingdom', 329, True)
('4cf551dec9af6dcbca23b17f', 'Sartori', 51.511784, -0.128021, '15-18 Great Newport St, London, Greater London, WC2H 7JE, United Kingdom', 344, True)
('4ae204bdf964a520d88921e3', 'Spaghetti House', 51.508592077526856, -0.13191928314995977, '66 Haymarket, London, Greater London, SW1Y 4RF, United Kingdom', 329, True)
('5345153a498ebfb423d09a53', 'The Whitehall', 51.50621265382943, -0.1270781828697975, '43 Whitehall, London, Greater London, SW1A 2BX, United Kingdom', 289, True)
('4c62d713ec94a5932e3d2aca', 'Prezzo Italian Restaurant London St Martins Lane', 51.50940280181761, -0.12712713136183257, "116 Saint Martin's Ln, London, WC2N 4BD, United Kingdom", 279, True)
('4b1d6873f964a5201d1024e3', 'Prezzo', 51.507386285970306, -0.12660951183383057, '31-32 Northumberland Av

In [195]:
print('Restaurants around location')
print('---------------------------')
for i in range(90, 99):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 91: Dishoom, Gaucho, Yeni, Flat Iron, Kiln, Nopi, The Araki, Cecconi's
Restaurants around location 92: Brasserie Blanc, Wahaca, Skylon, Spiritland, KORRITO, wagamama, Horn OK Please, Terrace Bar & Food
Restaurants around location 93: Brasserie Blanc, Wahaca, Skylon, Spiritland, KORRITO, wagamama, Horn OK Please, Terrace Bar & Food
Restaurants around location 94: Flat Iron, Kanada-Ya, Hoppers, Ceviche Soho, The Barbary, 10 Greek Street, eat TOKYO, Chotto Matte
Restaurants around location 95: Brasserie Blanc, Wahaca, Skylon, Spiritland, KORRITO, wagamama, Terrace Bar & Food, Horn OK Please
Restaurants around location 96: The Delaunay, Roka, Spring, San Carlo Cicchetti, The Lebanese Bakery, Christopher's, SUSHISAMBA, On The Bab
Restaurants around location 97: The Delaunay, Roka, SUSHISAMBA, San Carlo Cicchetti, The Lebanese Bakery, Barrafina, Christopher's, Spring
Restaurants around location 98: Barrafina,

In [196]:
map_London = folium.Map(location=[latitude, longitude], zoom_start=13)
folium.Marker(location=[latitude, longitude], popup='London').add_to(map_London)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_London)
map_London